In [ ]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled, filename = [2005, 2021, False, True, 'df-light.pkl']
else:
    start_year, end_year, chk, sampled, filename = [2005, 2021, True, False, 'df-full.pkl']

from sklearn.model_selection import train_test_split

df = pd.read_pickle(f'./{filename}')
data = df.iloc[:, 1:]
target = df['grav']

data = data.drop(columns=['an'],axis=1)
data = data[['catv', 'agg', 'dep', 'col', 'catr', 'catu', 'trajet', 'locp', 'circ', 'situ', 'lum', 'age_cls']]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)
# X_train, y_train, X_test, y_test = pt.train_test_split_along_time(data, target, 2018)

In [ ]:
import time
from my_libs.encoder_custom import EncoderCustom

start_time = time.time()

# cols_target_encoded = ['dep', 'age']
cols_target_encoded = []
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)

encoder = EncoderCustom(cols_target_encoded=cols_target_encoded, cols_onehot_encoded=cols_onehot_encoded)

X_train_enc, y_train_enc = encoder.transform(X_train, y_train, 'Train')
X_test_enc,  y_test_enc  = encoder.transform(X_test,  y_test,  'Test')

print("--- Features encoding performed in %s seconds ---" % (time.time() - start_time))

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

# params_dt = {'max_depth': 15, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_split': 3}
params_dt = {'max_depth': 11, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 4}

model_type = 'DecisionTreeClassifier'

evaluator = ModelEvaluator(model_type=model_type, params=params_dt, X_train=X_train_enc, y_train=y_train_enc, X_test=X_test_enc, y_test=y_test_enc)
model = evaluator.evaluate()

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

# params_rf_old = {'n_estimators': 34, 'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2}
# params_rf = {'n_estimators': 57, 'criterion': 'entropy', 'max_depth': 39, 'min_samples_split': 4}
params_rf = {'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 50}
model_type = 'RandomForestClassifier'

evaluator = ModelEvaluator(model_type=model_type, params=params_rf, X_train=X_train_enc, y_train=y_train_enc, X_test=X_test_enc, y_test=y_test_enc)
model = evaluator.evaluate()

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

best_params = {'learning_rate': 0.22, 'n_estimators': 51}
model_type = 'GradientBoostingClassifier'

evaluator = ModelEvaluator(model_type=model_type, params=best_params, X_train=X_train_enc, y_train=y_train_enc, X_test=X_test_enc, y_test=y_test_enc)
model = evaluator.evaluate()

In [ ]:
from shapash import SmartExplainer
import shap
shap.initjs()

xpl = SmartExplainer(
    model=model,
    # features_dict=house_dict,  # Optional parameter
    # preprocessing=encoder, # Optional: compile step can use inverse_transform method
    # postprocessing=postprocess # Optional: see tutorial postprocessing
)

y_pred = model.predict(X_test)

compile_df = X_test
compile_df['y_pred'] = y_pred

xpl.compile(
    x=compile_df.drop(columns=['y_pred'], axis=1),
    y_pred=compile_df['y_pred'], # Optional: for your own prediction (by default: model.predict)
    y_target=pd.Series(y_test), # Optional: allows to display True Values vs Predicted Values
)

app = xpl.run_app()